<a href="https://colab.research.google.com/github/LEFT-BEE/KMU-Algorithm/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B3%BC%ED%95%99/Softmax_Regression_Pratice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Softmax Regression

In [9]:
import torch
x_train = torch.FloatTensor([ [1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5],
 [1,2,5,6], [1,6,6,6], [1,7,7,7] ])
y_train = torch.FloatTensor([ [0,0,1], [0,0,1], [0,0,1], [0,1,0], [0,1,0], [0,1,0],
 [1,0,0], [1,0,0] ])

In [10]:
# W,b 초기화 
#optimizer 생성
W = torch.zeros(4, 3, requires_grad=True)
b = torch.zeros(1, 3, requires_grad=True)
optimizer = torch.optim.Adam([W,b], lr=0.1)

$H(x) = S(x^Tw +b)$

$Cost(w,b) = \frac{1}{n}\sum^m_{i=1} C(y_i , H(x_i))$

$C(y,\hat{y}) = -\sum^d_{j=1} y_j log(\hat{y}_j) $ 
-> 크로스 엔트로피 j는 배치데이터의 수 y는 실제 데이터 hat(y)는 예측값

In [16]:
# 반복횟수 설정, 가설 및 비용 설정
for epoch in range(3001):
 #hypothesis = torch.softmax(torch.mm(x_train, W)+b, dim=1)
 #cost = -torch.mean(torch.sum(y_train * torch.log(hypothesis), dim=1))

 hypothesis = (torch.mm(x_train, W)+b).softmax(dim=1)
 #softmax 는 sogmoid에 비해 유연하게 학습이 가능함
 cost = -(y_train * torch.log(hypothesis)).sum(dim=1).mean()
 #tensor연산의 *은 일반적인 곱셈이 아닌 행렬곱을 의미한다
 optimizer.zero_grad()
 cost.backward()
 optimizer.step()

 if epoch % 300 == 0:
  print("epoch: {}, cost: {:.6f}".format(epoch, cost.item()))

epoch: 0, cost: 0.002582
epoch: 300, cost: 0.002092
epoch: 600, cost: 0.001712
epoch: 900, cost: 0.001412
epoch: 1200, cost: 0.001173
epoch: 1500, cost: 0.000979
epoch: 1800, cost: 0.000822
epoch: 2100, cost: 0.000692
epoch: 2400, cost: 0.000585
epoch: 2700, cost: 0.000496
epoch: 3000, cost: 0.000421


In [7]:
#infer 
W.requires_grad_(False)
b.requires_grad_(False)
x_test = torch.FloatTensor([[1,11,10,9], [1,3,4,3], [1,1,0,1]])
test_all = torch.softmax(torch.mm(x_test, W)+b, dim=1)
print(test_all)
print(torch.argmax(test_all, dim=1))
#리스트값중 가장 큰 값을 출력 -> softmax의 값이 높을 수록 해당 레이블이 정답 이기에 스코어함수라고 부르기도함

tensor([[1.0000e+00, 2.8566e-25, 0.0000e+00],
        [7.2722e-03, 7.5765e-01, 2.3508e-01],
        [2.9427e-44, 6.3150e-15, 1.0000e+00]])
tensor([0, 1, 2])


## 조금더 깔끔하게 Softmax

1.레이블을 간단하게 입력받고 싶음 

ex) [1,0,0], [0,1,0], [0,0,1] -> 0, 1, 2

2.가설함수와 활성화 함수를 매번 구현하기 어려움

ex) hypothesis = torch.softmax(torch.mm(x_train, W)+b, dim=1)

cost = -torch.mean(torch.sum(y_train * torch.log(hypothesis), dim=1)) 와 같은 함수들

In [23]:
# pytoch의 functional 라이브러리를 F로 사용하겠다 선언
import torch.nn.functional as F
import torch.nn as nn

x_train = torch.FloatTensor([ [1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5],
 [1,2,5,6], [1,6,6,6], [1,7,7,7] ])
#문제 1번 해결 64비트의 부호 있는 정수는 torch.LongTensor를 사용한다.
y_train = torch.LongTensor([2,2,2,1,1,1,0,0])

model = nn.Linear(4,3)
optimizer = torch.optim.Adam(model.parameters(),lr=1)

In [24]:
for epoch in range(3001): 
 z = model(x_train)
 #생성한 모델에 학습데이터 입력
 cost = F.cross_entropy(z, y_train)
 #F.cross_entropy는 softamx와 cross_entropy가 합쳐진 것!
 optimizer.zero_grad()
 cost.backward()
 optimizer.step()

 if epoch % 300 == 0:
  print("epoch: {}, cost: {:.6f}".format(epoch, cost.item()))

epoch: 0, cost: 2.762645
epoch: 300, cost: 0.028284
epoch: 600, cost: 0.011228
epoch: 900, cost: 0.006065
epoch: 1200, cost: 0.003805
epoch: 1500, cost: 0.002602
epoch: 1800, cost: 0.001880
epoch: 2100, cost: 0.001412
epoch: 2400, cost: 0.001089
epoch: 2700, cost: 0.000858
epoch: 3000, cost: 0.000687


## Softmax Regression with Sklearn

sklearn 에서는 logisticRegression이 다중분류 문제에서 Softmax regression으로 구현됨


In [27]:
import numpy as np
from sklearn.linear_model import LogisticRegression

x_train = np.array([ [1,2,1,1], [2,1,3,2], [3,1,3,4], [4,1,5,5], [1,7,5,5],
 [1,2,5,6], [1,6,6,6], [1,7,7,7] ])
y_train = np.array([ 2, 2, 2, 1, 1, 1, 0, 0 ])

logistic = LogisticRegression() #모델생성 
logistic.fit(x_train, y_train) #train_data와 label을 넣어줌 for문으로 학습하는 과정이 fit함수하나로 요약됨 


pred = logistic.predict([[1,11,10,9], [1,3,4,3], [1,1,0,1]]) #추론단계
print(pred)
#입력에대한 출력값이 잘 나온다.

[0 2 2]
